In [1]:
%run '01_Algebraic_Structure.ipynb'

In [302]:
class Cartan_Group ( object ) :

    ADMISSIBLE_CARTAN_FAMILIES = { 'A' : ( 1 , +infinity ) ,
                                   'B' : ( 2 , +infinity ) ,
                                   'C' : ( 2 , +infinity ) ,
                                   'D' : ( 3 , +infinity ) ,
                                   'E' : ( 6 , 8 ) ,
                                   'F' : ( 4 , 4 ) ,
                                   'G' : ( 2 , 2 )
                                 }


    def __add__ ( self , other:"Cartan_Group" ) -> "Irreducible_Cartan_Group" or "Direct_Sum_Of_Cartan_Groups" :
        """Returns the coproduct of two Cartan groups."""
        assert isinstance( other , type(self).__bases__[1] ) , 'The input for ``other`` need to be an object of the class '+str( type(self).__bases__[1] )+'.'
        New_Constituent_Parts = self.Constituent_Parts() + other.Constituent_Parts()
        if len(New_Constituent_Parts) == 1 :
            Single_Part = New_Constituent_Parts[0]
            return Single_Part
        else :
            return Direct_Sum_Of_Cartan_Groups( New_Constituent_Parts )


    def Cartan_Type ( self ) -> CartanType:
        """Returns the Cartan type."""
        return CartanType( self.Cartan_String() )


    @staticmethod
    def Constructor ( *Input:tuple ) -> "Irreducible_Cartan_Group" or "Direct_Sum_Of_Cartan_Groups" :

        def Get_Cartan_Group_From_String ( String:str ) -> "Irreducible_Cartan_Group" or "Direct_Sum_Of_Cartan_Groups" :
            Summands = []
            for String_Part_Counter , String_Part in enumerate( String.split('x') , start=1 ) :
                Cartan_Family = String_Part[0]
                try : Cartan_Degree = int(String_Part[1:])
                except : raise ValueError('The algorithm is not able to extract a Cartan degree from the '+str(String_Part_Counter)+'-th string part `'+String_Part+'`.')
                Summands += [ Irreducible_Cartan_Group( Cartan_Family , Cartan_Degree ) ]

            if len(Summands) == 1 :
                Single_Summand = Summands[0]
                return Single_Summand
            else :
                return Direct_Sum_Of_Cartan_Groups( Summands )

        def Get_Cartan_Group_From_2Tuple ( Tuple:tuple ) -> "Irreducible_Cartan_Group" :
            assert len(Tuple) == 2 , \
                   ValueError('The algorithm can not extract Cartan data from a tuple of length '+str(len(Tuple))+'; it expects a tuple of length 2.')

            assert type(Tuple[0]) == str , \
                   TypeError('The algorithm can not extract Cartan family from the first entry of the tuple: Tuple[0] = '+str(Tuple[0])+'.')
            Cartan_Family = Tuple[0]

            assert Tuple[1] in ZZ , \
                   TypeError('The algorithm can not extract Cartan degree from the second entry of the tuple: Tuple[1] = '+str(Tuple[1])+'.')
            Cartan_Degree = Tuple[1]

            return Irreducible_Cartan_Group( Cartan_Family , Cartan_Degree )

        if   len(Input) == 0 :
            return Direct_Sum_Of_Cartan_Groups( [] )

        elif len(Input) == 1 :
            Input = Input[0]

            if   Input in [ None , 0 , "" ] :
                return Direct_Sum_Of_Cartan_Groups( [] )

            elif type(Input) == str :
                return Get_Cartan_Group_From_String( Input )

            elif type(Input) == list :
                Result = Direct_Sum_Of_Cartan_Groups( [] )
                for Entry_Counter , Entry in enumerate( Input , start=1 ) :
                    if   type(Entry) == str                         : Result += Get_Cartan_Group_From_String( Entry )
                    elif type(Entry) == tuple                       : Result += Get_Cartan_Group_From_2Tuple( Entry )
                    elif type(Entry) == Irreducible_Cartan_Group    : Result += Entry
                    elif type(Entry) == Direct_Sum_Of_Cartan_Groups : Result += Entry
                    elif Entry in [ None , 0 , "" ]                 : pass
                    else                                            : raise TypeError('The '+str(Entry_Counter)+'-th entry of the input is of an inappropriate type.')
                if Result.Is_Irreducible() : return Result[0]
                else                       : return Result

        elif len(Input) == 2 :
            if type(Input[0]) == str and type(Input[1]) == int : return Get_Cartan_Group_From_2Tuple( *Input )
            else                                               : return Cartan_Group.Constructor( list(Input) )

        else :
            return Cartan_Group.Constructor( list(Input) )


    # Synonyms for the method ``Weyl_Character``.
    def rmV ( self , Highest_Weight:"Weight" ) -> "Weyl_Character" :
        """Returns the Weyl character associated to given highest weights."""
        return self.Weyl_Character( Highest_Weight=Highest_Weight )


    def Weyl_Character ( self , Highest_Weight:"Weight" ) -> "Weyl_Character" :
        """Returns the Weyl character associated to a given highest weight."""
        WCR = self.Weyl_Character_Ring()
        return WCR( Highest_Weight )


    def Weyl_Character_Ring ( self , Style:str='coroots' ) -> "Weyl_Character_Ring" :
        """Returns the Weyl character ring associated to the Cartan type of G."""
        return WeylCharacterRing( self.Cartan_Type() , style=Style )




In [303]:
class Irreducible_Cartan_Group ( Irreducible_Algebraic_Structure , Cartan_Group ) :

    def __init__ ( self , Cartan_Family:str , Cartan_Degree:int ) -> None :
        """
        Initialize an irreducible algebraic group of given cartan type.

        INPUT:
        - ``Cartan_Family`` -- str;
        - ``Cartan_Degree`` -- int;

        OUTPUT: None.
        """
        assert Cartan_Family in self.ADMISSIBLE_CARTAN_FAMILIES.keys() , \
               ValueError('The input for ``Cartan_Degree`` need to be a letter from the alphabet '+str(self.ADMISSIBLE_CARTAN_FAMILIES.keys())+'.')
        self._Cartan_Family = Cartan_Family

        assert Cartan_Degree in ZZ , \
               ValueError('The input for ``Cartan_Degree`` need to be an integer.')
        Lower_Bound , Upper_Bound = self.ADMISSIBLE_CARTAN_FAMILIES[Cartan_Family]
        assert Lower_Bound <= Cartan_Degree and Cartan_Degree <= Upper_Bound , \
               ValueError('If the Cartan family is '+str(Cartan_Family)+', then the input for ``Cartan_Degree`` need to between '+str(Lower_Bound)+' and '+str(Upper_Bound)+'.')
        self._Cartan_Degree = Cartan_Degree


    def __repr__ ( self ) -> ( str , int ) :
        """Returns all attributes which are necessary to initialize the object."""
        return self.Cartan_Family() , self.Cartan_Degree()


    def __str__ ( self ) :
        """Returns a one-line string as short description."""
        return 'Irreducible Cartan Group of type '+str(self.Cartan_String())+'.'


    def __truediv__ ( self , other:"Parabolic_Subgroup_In_Irreducible_Cartan_Group" ) -> "Irreducible_Homogeneous_Variety" :
        """Returns the homogeneous variety G/P."""
        assert type( other ) == Parabolic_Subgroup_In_Irreducible_Cartan_Group , \
               TypeError('The divisor need to be a parabolic subgroup.')
        assert self == other.Parent_Group() , \
               ValueError('The parabolic subgroup (divisor) need to have ``self`` as parent group.')
        return Irreducible_Homogeneous_Variety( other )


    def Borel_Subgroup( self ) -> "Parabolic_Subgroup_In_Irreducible_Cartan_Group" :
        """Returns the Borel subgroup of G (= minimal parabolic, i.e. no nodes are included)."""
        return self.Parabolic_Subgroup( Included_Nodes=set({}) , Excluded_Nodes=None )


    def Cartan_Degree ( self ) -> int :
        """Returns the attribute ``_Cartan_Degree``."""
        return self._Cartan_Degree


    def Cartan_Family ( self ) -> str :
        """Returns the attribute ``_Cartan_Family``."""
        return self._Cartan_Family


    def Cartan_String ( self ) -> str :
        """Returns the Cartan string."""
        return self.Cartan_Family()+str(self.Cartan_Degree())


    def Dimension ( self ) -> int :
        """
        Return the dimension of ``self`.

        INPUT:
        - ``self`` -- Cartan_Group; the Cartan group G.

        OUTPUT:
        - ``Output`` -- Integer; the dimension of G

        ALGORITHM:
        Thanks to the post by Pieter Belmans concerning the dimension of partial flag varieties
        from Jun 14th, 2017 on his blog (cf. to [Blog_PieterBelmans]_). The link is
        https://pbelmans.ncag.info/blog/2017/06/14/dimensions-of-partial-flag-varieties/
        (Date: Apr 21st, 2021).

        For the Borel group B ⊂ G: dim B = # of positive roots + rank (which accounts for the center)
        For G: dim G = # of roots in the root system + rank (which accounts for the center)
            i.e. # of roots in the root system = # of positive roots + # negative roots
            and # of positive roots = # negative roots
            so # of roots in the root system = 2 * # of positive roots

        REFERENCE:
        [Blog_PieterBelmans] https://pbelmans.ncag.info/blog/
        """
        Rank = self.Cartan_Type().dynkin_diagram().rank()
        Number_Of_Positive_Roots = len( list( self.Cartan_Type().root_system().root_lattice().positive_roots() ) )
        return Rank + 2 * Number_Of_Positive_Roots


    def Is_Exceptional ( self ) -> bool :
        """Test if the Cartan family of ``self`` is out of [ E , F , G ]."""
        return self.ADMISSIBLE_CARTAN_FAMILIES[self.Cartan_Family()][1] < +infinity


    def Is_Ordinary ( self ) -> bool :
        """Test if the Cartan family of ``self`` is out of [ A , B , C , D ]."""
        return not self.Is_Exceptional()


    def Maximal_Parabolic_Subgroup ( self , Excluded_Node :int ) -> "Parabolic_Subgroup_In_Irreducible_Cartan_Group" :
        """
        Returns the maximal parabolic subgroup associated to the excluded node.
        It is the parabolic subgroup obtained by adding all neative roots except the ``ExcludedNode``-th one.
        """
        Nodes = set( self.Cartan_Type().index_set() )
        assert Excluded_Node in Nodes , \
               ValueError('The excluded node neet to be an element of the set of all nodes ' + str(Nodes) + '.')
        return self.Parabolic_Subgroup( Included_Nodes=None , Excluded_Nodes={ Excluded_Node } )


    def Parabolic_Subgroup ( self , Included_Nodes :set[ int ] or None =None , Excluded_Nodes :set[ int ] or None =None ) -> "Parabolic_Subgroup_In_Irreducible_Cartan_Group" :
        """Returns the parabolic subgroup associated to the included nodes.

        INPUT:
        - ``self`` -- Irreducible_Cartan_Group; the Cartan group G.
        - ``Included_Nodes``  -- set or None (default: None);
        - ``Excluded_Nodes``  -- set or None (default: None);

        OUTPUT:
        - ``Output`` -- Parabolic subgroup; the parabolic subgroup P ⊂ G associated to the included nodes.

        .. NOTE::
        The Borel subgroup B ⊂ G is the minimal parabolic subgroup obtained by all positive roots and the center.
        Any further parabolic subgroup is an extension of B by adding a certain amount of negative roots.
        So ``Included_Nodes`` is the data which negative roots are taken into account.
        """
        Nodes = set( self.Cartan_Type().index_set() )

        # Test the input for ``Included_Nodes`` and ``Excluded_Nodes``
        if   Included_Nodes == None and Excluded_Nodes == None :
            raise ValueError('There need to be an input for either ``Included_Nodes`` or ``Excluded_Nodes``.')

        elif type(Included_Nodes) == set and Excluded_Nodes == None :
            assert Included_Nodes.issubset( Nodes ) , \
                   ValueError('The set of included nodes need to be a subset of set of all nodes ' + str( Nodes ) + '.')

        elif Included_Nodes == None and type(Excluded_Nodes) == set :
            assert Excluded_Nodes.issubset( Nodes ) , \
                   ValueError('The set of excluded nodes need to be a subset of set of all nodes ' + str( Nodes ) + '.')
            Included_Nodes = Nodes.difference( Excluded_Nodes )

        elif type(Included_Nodes) == set and type(Excluded_Nodes) == set :
            assert Included_Nodes.union(Excluded_Nodes) == Nodes , \
                   ValueError('The union of included and excluded nodes need to be a the set of all nodes ' + str( Nodes ) + '.')
            assert Included_Nodes.intersection(Excluded_Nodes) == set([]) , \
                   ValueError('The intersection of included and excluded nodes need to be empty.')

        else :
            raise ValueError('The inputs for ``Included_Nodes`` and ``Excluded_Nodes`` are of inappropriate values.')

        return Parabolic_Subgroup_In_Irreducible_Cartan_Group( Parent_Group=self , Included_Nodes=Included_Nodes )


In [305]:
class Direct_Sum_Of_Cartan_Groups ( Direct_Sum_Of_Algebraic_Structures , Cartan_Group ) :

    def __init__ ( self , Summands:list[Irreducible_Cartan_Group] ) -> None :
        """
        Initialize an algebraic group of given cartan type.

        INPUT:
        - ``Input`` -- list[Irreducible_Cartan_Group];

        ..ToDo: Do not initalize if len == 0 or == 1.

        OUTPUT: None.
        """
        self._Summands = []
        for Summand_Counter , Summand in enumerate( Summands , start=1 ) :
            assert type(Summand) == Irreducible_Cartan_Group , \
                   TypeError('The '+str(Summand_Counter)+'-th component need to be an object of the class ``Irreducible_Cartan_Group``.')
            self._Summands += [ Summand ]


    def __repr__ ( self ) -> list[ Irreducible_Cartan_Group ] :
        """Returns all attributes which are necessary to initialize the object."""
        return self.Summands()


    def __str__ ( self ) :
        """Returns a one-line string as short description."""
        if len(self) == 0 : return 'Trivial group.'
        else              : return 'Direct sum of Cartan groups of type '+str(self.Cartan_String())+'.'


    def Cartan_Degree ( self ) -> list[int] :
        """Returns the Cartan degree of each component of ``self``."""
        return [ Summand.Cartan_Degree() for Summand in self.Summands() ]


    def Cartan_Family ( self ) -> list[str] :
        """Returns the Cartan family of each component of ``self``."""
        return [ Summand.Cartan_Family() for Summand in self.Summands() ]


    def Cartan_String ( self ) -> str :
        """Returns the Cartan string."""
        return 'x'.join([ Summand.Cartan_String() for Summand in self.Summands() ])


    def Dimension ( self ) -> int :
        """Return the dimension of ``self`."""
        return sum([ Summand.Dimension() for Summand in self.Summands() ])


    def Is_Exceptional ( self ) -> list[bool] :
        """Test if the Cartan family of each component of ``self`` is out of [ E , F , G ]."""
        return [ Summand.Is_Exceptional() for Summand in self.Summands() ]


    def Is_Ordinary ( self ) -> list[bool] :
        """Test if the Cartan family of each component of ``self`` is out of [ A , B , C , D ]."""
        return [ Summand.Is_Ordinary() for Summand in self.Summands() ]
